# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

In [ ]:
#all other imports 


In [16]:
#useful variables
addtional_customer_cols = ['PRODUCT_GROUP', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE']

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#observe warning above - 5 columns have mixed types.  Then need to be identified and addressed

In [28]:
#check the correct rows and cols are loaded
#make a copy of the df
print("azdias: ",azdias.shape)
print("customers: ",customers.shape)

azdias_org = azdias
customers_org = customers

azdias:  (891221, 366)
customers:  (191652, 369)


In [4]:
azdias.head

<bound method NDFrame.head of            LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0       910215        -1         NaN       NaN          NaN          NaN   
1       910220        -1         9.0       0.0          NaN          NaN   
2       910225        -1         9.0      17.0          NaN          NaN   
3       910226         2         1.0      13.0          NaN          NaN   
4       910241        -1         1.0      20.0          NaN          NaN   
5       910244         3         1.0      10.0          NaN          NaN   
6       910248        -1         9.0       0.0          NaN          NaN   
7       910261        -1         1.0      14.0          NaN          NaN   
8       645145        -1         9.0      16.0          NaN          NaN   
9       645153        -1         5.0      17.0          NaN          NaN   
10      645165         0         1.0      10.0          NaN          NaN   
11      645169        -1         NaN       NaN          Na

In [5]:
customers.head

<bound method NDFrame.head of            LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0         9626         2         1.0      10.0          NaN          NaN   
1         9628        -1         9.0      11.0          NaN          NaN   
2       143872        -1         1.0       6.0          NaN          NaN   
3       143873         1         1.0       8.0          NaN          NaN   
4       143874        -1         1.0      20.0          NaN          NaN   
5       143888         1         1.0      11.0          NaN          NaN   
6       143904         2         1.0      10.0          NaN          NaN   
7       143910         1         1.0      10.0          NaN          NaN   
8       102160         2         3.0       5.0          NaN          NaN   
9       102173         1         1.0      20.0          NaN          NaN   
10      102184        -1         7.0      14.0          NaN          NaN   
11      102185         1         1.0      10.0          Na

In [6]:
#load the values and attributes
dias_values = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header =1)
dias_attribs = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header = 1)


In [7]:
dias_values.head

<bound method NDFrame.head of                 Attribute                                        Description  \
NaN              AGER_TYP                                 best-ager typology   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN  ALTERSKATEGORIE_GROB       age classification through prename analysis    
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN                   NaN                                                NaN   
NaN       

In [8]:
dias_attribs.head

<bound method NDFrame.head of     Information level                    Attribute  \
NaN               NaN                     AGER_TYP   
NaN            Person         ALTERSKATEGORIE_GROB   
NaN               NaN                    ANREDE_KZ   
NaN               NaN                CJT_GESAMTTYP   
NaN               NaN            FINANZ_MINIMALIST   
NaN               NaN                FINANZ_SPARER   
NaN               NaN             FINANZ_VORSORGER   
NaN               NaN               FINANZ_ANLEGER   
NaN               NaN        FINANZ_UNAUFFAELLIGER   
NaN               NaN             FINANZ_HAUSBAUER   
NaN               NaN                    FINANZTYP   
NaN               NaN                  GEBURTSJAHR   
NaN               NaN              GFK_URLAUBERTYP   
NaN               NaN             GREEN_AVANTGARDE   
NaN               NaN                   HEALTH_TYP   
NaN               NaN          LP_LEBENSPHASE_FEIN   
NaN               NaN          LP_LEBENSPHASE_GROB  

In [13]:
#find the cols with mixed data types
def findColumnsWithMixedDataTypes(df):
    '''
    '''
    columnsWithMixedDataTypes = []
    for col in df.columns:
        weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis=1)
        if len(df[weird]) > 0:
            #print(col)
            columnsWithMixedDataTypes.append(col)
    
    return columnsWithMixedDataTypes

In [14]:
azdias_mixeddatatypes_cols = findColumnsWithMixedDataTypes(azdias)
print(azdias_mixeddatatypes_cols)

['CAMEO_DEU_2015', 'CAMEO_DEUG_2015', 'CAMEO_INTL_2015', 'D19_LETZTER_KAUF_BRANCHE', 'EINGEFUEGT_AM', 'OST_WEST_KZ']


In [15]:
customers_mixeddatatypes_cols = findColumnsWithMixedDataTypes(customers)
print(customers_mixeddatatypes_cols)

['CAMEO_DEU_2015', 'CAMEO_DEUG_2015', 'CAMEO_INTL_2015', 'D19_LETZTER_KAUF_BRANCHE', 'EINGEFUEGT_AM', 'OST_WEST_KZ']


In [ ]:
#Conclusion: the same columns have mixed types in both files => consistency

In [17]:
#look at the distinct values in the these columns

#Error message on loading: DtypeWarning: Columns (18,19) have mixed types
#=> these are the only two we need to worry about
#first check do they appear in the lists above
azdias_mixed_colnames = azdias.columns[18:20]
customers_mixed_colnames = customers.columns[18:20]
print (azdias_mixed_colnames)
print (customers_mixed_colnames)



Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')
Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')


In [39]:
#yes, cols do appear in the list above
mixed_dtype_colnames = azdias_mixed_colnames
#check unique values in those columns
pd.value_counts(azdias.CAMEO_DEUG_2015)

 8    134441
 9    108177
 6    105874
 4    103912
-1     99352
 3     86779
 2     83231
 7     77933
 5     55310
 1     36212
Name: CAMEO_DEUG_2015, dtype: int64

In [25]:
pd.value_counts(azdias.CAMEO_INTL_2015)

51      77576
51.0    56118
41      53459
24      52882
41.0    38877
24.0    38276
14      36524
43      32730
14.0    26360
54      26207
43.0    23942
25      22837
54.0    19184
22      19173
25.0    16791
23      15653
13      15272
45      15206
22.0    13982
55      13842
52      11836
23.0    11097
13.0    11064
31      11041
45.0    10926
34      10737
55.0    10113
15       9832
52.0     8706
44       8543
31.0     7983
34.0     7787
12       7645
15.0     7142
44.0     6277
35       6090
32       6067
33       5833
12.0     5604
32.0     4287
35.0     4266
33.0     4102
XX        373
Name: CAMEO_INTL_2015, dtype: int64

In [27]:
pd.value_counts(customers.CAMEO_DEUG_2015)

2      17574
4      16458
6      14008
3      13585
1      12498
8       9716
5       8624
7       7878
2.0     5910
4.0     5606
3.0     4805
9       4731
6.0     4709
1.0     4280
8.0     3333
5.0     3042
7.0     2680
9.0     1661
X        126
Name: CAMEO_DEUG_2015, dtype: int64

In [26]:
pd.value_counts(customers.CAMEO_INTL_2015)

14      14708
24      13301
41       8461
43       7158
25       6900
15       6845
51       5987
13       5728
22       5566
14.0     4939
24.0     4504
23       4276
34       3945
45       3936
54       3537
41.0     2859
55       2794
12       2791
43.0     2476
25.0     2472
15.0     2372
44       2144
51.0     2126
31       2050
13.0     1955
22.0     1941
35       1741
23.0     1494
34.0     1423
45.0     1352
54.0     1258
32       1256
33       1178
12.0      924
55.0      920
52        770
44.0      688
31.0      681
35.0      553
32.0      440
33.0      396
52.0      253
XX        126
Name: CAMEO_INTL_2015, dtype: int64

In [37]:
#method to update these two columns
#NB CAMEO_INTL_2015 is CAMEO_DEINTL_2015 in DIAS Attribute Values 2017 s/s 
#CAMEO_INTL_2015: object type can be set to -1
#CAMEO_DEUG_2015: object type can be set to -1
#also will cast ALL values to ints

def updateMixedDataTypeColumns(df, list_of_cols):
    '''
    '''
    #first replace the object types with -1
    for col in list_of_cols:
        df[col].replace({"X": -1, "XX": -1}, inplace=True)
        #set Nan to -1
        df[col] = df[col].fillna(-1)
        #cast everything to int
        df[col] = df[col].astype(int)
    #return new df
    

In [38]:
updateMixedDataTypeColumns(azdias, mixed_dtype_colnames)

In [9]:
#look at the numeric attributes #1
num_attributes = customers.select_dtypes(exclude='object')
num_attributes.describe()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
count,191652.000000,191652.000000,145056.000000,145056.000000,11766.000000,5100.000000,1275.000000,236.000000,139810.000000,141725.000000,...,143781.000000,143781.000000,143781.000000,137910.000000,145056.000000,141725.000000,191652.000000,191652.000000,191652.000000,191652.000000
mean,95826.500000,0.344359,1.747525,11.352009,12.337243,13.672353,14.647059,15.377119,10.331579,4.965863,...,4.374417,4.564769,3.168868,4.152716,8.646371,3.723133,2.576806,0.090247,1.376432,3.060907
std,55325.311233,1.391672,1.966334,6.275026,4.006050,3.243335,2.753787,2.307653,4.134828,14.309694,...,2.924355,2.887035,2.233516,1.974375,1.154001,2.095540,1.168486,0.286536,0.484492,1.086254
min,1.000000,-1.000000,1.000000,0.000000,2.000000,2.000000,5.000000,8.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,47913.750000,-1.000000,1.000000,8.000000,9.000000,11.000000,13.000000,14.000000,9.000000,1.000000,...,2.000000,2.000000,1.000000,2.000000,9.000000,2.000000,1.000000,0.000000,1.000000,3.000000
50%,95826.500000,0.000000,1.000000,11.000000,13.000000,14.000000,15.000000,16.000000,10.000000,1.000000,...,4.000000,4.000000,3.000000,5.000000,9.000000,3.000000,3.000000,0.000000,1.000000,3.000000
75%,143739.250000,2.000000,1.000000,16.000000,16.000000,16.000000,17.000000,17.000000,13.000000,4.000000,...,7.000000,7.000000,4.000000,6.000000,9.000000,5.000000,3.000000,0.000000,2.000000,4.000000
max,191652.000000,3.000000,9.000000,21.000000,18.000000,18.000000,18.000000,18.000000,25.000000,523.000000,...,11.000000,13.000000,11.000000,6.000000,9.000000,8.000000,6.000000,1.000000,2.000000,9.000000


In [15]:
#look at the numeric attributes #2
num_attributes.isna().sum().sort_values(ascending=False).head()

ALTER_KIND4     191416
ALTER_KIND3     190377
ALTER_KIND2     186552
ALTER_KIND1     179886
KK_KUNDENTYP    111937
dtype: int64

In [16]:
#look at non-numeric attributes #1
non_num_attributes = customers.select_dtypes(include='object')
non_num_attributes.describe()

,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ,PRODUCT_GROUP,CUSTOMER_GROUP
count,141224,141224,141224,143955,141725,141725,191652,191652
unique,45,19,43,35,3034,2,3,2
top,2D,2,14,D19_UNBEKANNT,1992-02-10 00:00:00,W,COSMETIC_AND_FOOD,MULTI_BUYER
freq,11208,17574,14708,31910,64744,130382,100860,132238


In [17]:
#look at non-numeric attributes #2
non_num_attributes.head
#non_num_attributes.describe()

<bound method NDFrame.head of        CAMEO_DEU_2015 CAMEO_DEUG_2015 CAMEO_INTL_2015  \
0                  1A               1              13   
1                 NaN             NaN             NaN   
2                  5D               5              34   
3                  4C               4              24   
4                  7B               7              41   
5                  5D               5              34   
6                  3B               3              23   
7                  1D               1              15   
8                  9E               9              55   
9                  1D               1              15   
10                NaN             NaN             NaN   
11                 2D               2              14   
12                 4A               4              22   
13                 6B               6              43   
14                 9D               9              51   
15                 1D               1              15   
1

In [18]:
non_num_attributes.isna().sum().sort_values(ascending=False).head()

CAMEO_INTL_2015    50428
CAMEO_DEUG_2015    50428
CAMEO_DEU_2015     50428
OST_WEST_KZ        49927
EINGEFUEGT_AM      49927
dtype: int64

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')